In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
import numpy as np

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
import os
import sys

# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("gemini2.5.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

# Now you can import from GetXY.py
from GetXY import expressions_not_in_x, outsideExpr, longer_Exps, y_test, out_y_test, long_y_test

x_test = expressions_not_in_x
out_x_test = outsideExpr
long_x_test = longer_Exps
# ... rest of your code
print("Successfully imported variables!")

-5 + -2 + -2
2543
-9.0

Expressions not in x:
-2 - -2 + -2
True
1457
-2.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]
Successfully imported variables!


In [3]:
tokenizer = AutoTokenizer.from_pretrained("./output")


In [4]:
model_id = "google/gemma-3-270m-it"

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16,
    device_map="auto", # Automatically uses GPU if available
)

model = PeftModel.from_pretrained(base_model, "./output")

In [5]:
inputs = []

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("added padding")
for i in x_test:
    messages = [
        {"role": "user", "content": i}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs.append(tokenizer(prompt, return_tensors="pt").to(model.device))
print(inputs[0])

{'input_ids': tensor([[     2,      2,    105,   2364,    107, 236772, 236778,    753,    753,
         236778,    900,    753, 236778,    106,    107,    105,   4368,    107]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


In [7]:
responses = []
for input in inputs:
    output = model.generate(
        **input,
        max_new_tokens=150,  # The maximum number of new tokens to generate
        do_sample=True,      # Enables sampling for more creative responses
        temperature=0.7,     # Controls randomness (lower is more deterministic)
        top_k=50,            # Considers the top 50 tokens by probability
        top_p=0.95           # Uses nucleus sampling
    )
    responses.append(tokenizer.batch_decode(output, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for o

In [9]:
print(responses[0], "correct answer: ", y_test[0])

['user\n-2 - -2 + -2\nmodel\n-2.0'] correct answer:  -2.0


In [18]:
resp_nums = []
for i in responses:
    list = i[0].split("\nmodel\n")
    resp_num = np.float32(list[1])
    resp_nums.append(resp_num)



In [19]:
diffs = []
for i in range(len(y_test)):
    diff = y_test[i] - resp_nums[i]
    diffs.append(abs(diff))
print(np.mean(diffs))

1.240906
